In [95]:
from utils import read_jsonl, save_jsonl
import pandas as pd
from pydantic import BaseModel, model_validator, field_validator, Field, ValidationInfo
from typing import List, Dict, Union, Any, Optional
import instructor
from openai import OpenAI
import os
import json

In [96]:
client = instructor.patch(OpenAI(api_key=os.environ['OPENAI_API_KEY']))
MODEL = "gpt-3.5-turbo-0125"

# 🧠 Load data

- reference KBs
- predicted KBs
- Wikidata Properties by Usage Count

In [97]:
pred_kbs = read_jsonl('../../data/prediction.jsonl')
ref_kbs = read_jsonl('../../data/wikidata_entities.jsonl')
popular_properties = pd.read_csv('../../data/wikidata-properties-counts.csv')
print(len(popular_properties))
popular_properties.head()

3896


,ID,label,description,Data type[1],Counts[2],largest_number
0,P2860,cites work,citation from one creative or scholarly work t...,WI,"292,583,247 M 390 N",292583247
1,P1545,series ordinal,position of an item in its parent series (most...,S,"175,830,141 Q 2,298 N",175830141
2,P2093,author name string,stores unspecified author or editor name for p...,S,"138,055,438 M 589,477 Q 29,173 R 213 N",138055438
3,P31,instance of,that class of which this subject is a particul...,WI,"114,614,926 M 20 N",114614926
4,P248,stated in,to be used in the references field to refer to...,WI,"98,211,619 R 122 Q 19 N",98211619


In [100]:
ref_kbs[-1]['properties']

{'student of': ['Remigio dei Girolami', 'Brunetto Latini', "Cecco d'Ascoli"],
 'IMDb ID': ['nm0019604'],
 'Library of Congress authority ID': ['n78095495'],
 'VIAF ID': ['97105654'],
 'ISNI': ['0000000121446210'],
 'MusicBrainz artist ID': ['6c5a8114-cc72-43bc-a741-00756b317e0e'],
 'Commons category': ['Dante Alighieri'],
 'place of birth': ['Florence'],
 'place of death': ['Ravenna'],
 'GND ID': ['118523708'],
 'Bibliothèque nationale de France ID': ['118985852'],
 'IdRef ID': ['026811081'],
 'SBN author ID': ['CFIV008732'],
 'mother': ['Bella degli Abati'],
 'father': ['Alighiero di Bellincione'],
 'spouse': ['Gemma Donati'],
 'family': ['Alighieri'],
 'child': ['Jacopo Alighieri', 'Pietro Alighieri', 'Antonia Alighieri'],
 'image': ['Portrait de Dante.jpg'],
 'Find a Grave memorial ID': ['5101'],
 'Freebase ID': ['/m/028p0'],
 'NACSIS-CAT author ID': ['DA00360083'],
 'instance of': ['human'],
 "topic's main category": ['Category:Dante Alighieri'],
 'Commons gallery': ['Dante Alighie

In [32]:
ref_kbs[0].keys()

dict_keys(['entity_label', 'properties', 'chunked_content'])

In [33]:
pred_kbs[0].keys()

dict_keys(['entity_label', 'properties'])

# 🔎 Filter the Reference KB by the most popular Properties

This is a method to constrain the y label for our model so that the task is easier.

Popular properties are assigned by looking at their usage count across all of Wikidata. [This info is available here](https://www.wikidata.org/wiki/Wikidata:Database_reports/List_of_properties/all)

In [34]:
list(ref_kbs[0]['properties'].items())[-30:-15] 
# there's a lot of garbage here that isn't even interesting e.g Rotten Tomatoes ID
# but it's mixed in with the good stuff

[('Geni.com profile ID', ['6000000055367165869']),
 ('Reddit topic ID', ['alexei_navalny']),
 ('Radio France person ID', ['alexei-navalny']),
 ('WorldCat Entities ID', ['E39PBJhxkxBrf8HXdMBvGCfXh3']),
 ('Süddeutsche Zeitung topic ID', ['Alexej_Nawalny']),
 ('Der Spiegel topic ID', ['alexej_nawalny']),
 ('Douban movie celebrity ID', ['1392443']),
 ('Kinobox person ID', ['1613953']),
 ('Great Encyclopedia of Cyril and Methodius entry ID',
  ['Навальный Алексей Анатольевич']),
 ('patronym or matronym for this person', ['Anatolyevich']),
 ('WikiKids ID', ['Aleksej_Navalny']),
 ('date of death', ['+2024-02-16T00:00:00Z']),
 ('place of death', ['Corrective colony No. 3, YaNAO']),
 ('Gran Enciclopèdia Catalana ID (former scheme)', ['21139531']),
 ('Canadiana Name Authority ID', ['ncf13696330'])]

In [36]:
def filter_ref_kb_by_top_n_wikiproperties(ref_kbs, wikiproperties, N):

    filtered_ref_kbs = []
        
    for kb in ref_kbs:
        properties = kb['properties']

        top_N = wikiproperties.sort_values(by='largest_number', ascending=False).head(N)

        top_N = top_N['label'].tolist()

        kb['properties'] = {
            key: value for key, value in properties.items() if key in top_N
            }
        filtered_ref_kbs.append(kb)
    
    return filtered_ref_kbs


filtered_ref_kbs = filter_ref_kb_by_top_n_wikiproperties(ref_kbs, popular_properties, 200)
print(len(filtered_ref_kbs[0]['properties']))
filtered_ref_kbs[0]['properties']


36


{'member of': ['Russian Opposition Coordination Council',
  'Yale World Fellows'],
 'sex or gender': ['male'],
 'educated at': ['Finance University under the Government of the Russian Federation',
  'Yale University',
  "Peoples' Friendship University of Russia",
  'Yale World Fellows'],
 'image': ['Alexey Navalny (cropped) 1.jpg'],
 'member of political party': ['Yabloko',
  'Progress Party',
  'Russia of the Future'],
 'Commons category': ['Alexey Navalny'],
 'employer': ['Anti-Corruption Foundation', 'Aeroflot'],
 'date of birth': ['+1976-06-04T00:00:00Z'],
 'religion or worldview': ['Eastern Orthodoxy'],
 'country of citizenship': ['Soviet Union', 'Russia'],
 'field of work': ['politics', 'jurisprudence'],
 'place of birth': ['Butyn'],
 'instance of': ['human'],
 'described by source': ['Lentapedia', 'Navalny'],
 'official website URL': ['https://navalny.com'],
 'given name': ['Alexey'],
 'significant event': ['Yves Rocher case',
  'poisoning of Alexei Navalny',
  'Kirovles trial',

# 🪬 Define Evaluation Model

In [57]:

class ValidatedProperty(BaseModel):
    property_name: str
    property_value: Union[List[str], str, int, Dict[str, str]]
    is_valid: bool = Field(
      ...,
        description="""Whether the property value is valid or not, judged 
        against the reference knowledge base.""",
    )
    error_message: Optional[str] = Field(
        None, description="The error message if the value is not valid."
    )
    matching_reference_property: Optional[str] = Field(
        ...,
        description="""The corresponding property in the reference knowledge base which 
        this predicted property correctly answers."""
    )


class ValidatedKnowledgeBase(BaseModel):
    entity_label: str
    properties: List[ValidatedProperty] = Field(
        ...,
        description="A list of validations about whether the property is valid given the reference knowledge base.",
    )

    @field_validator('properties')
    @classmethod
    def assert_all_properties_validated(cls, validation_properties: List[ValidatedProperty], info: ValidationInfo):
        existing_pred_properties = info.context.get("existing_pred_properties")
        if len(validation_properties) != len(existing_pred_properties):
            raise ValueError(
                f"""Number of properties validated does not match number of properties in the prediction knowledge base. 
                Number of properties validated: {len(validation_properties)}, 
                Number of properties in the text: {len(existing_pred_properties)}"""
                )
        return validation_properties

    @field_validator('properties')
    @classmethod
    def assert_matching_ref_in_ref_kb(cls, validation_properties: List[ValidatedProperty], info: ValidationInfo):
        '''
        Make sure that the reference property that is linked to the prediction 
        actually exists in the reference knowledge base.
        '''
        existing_ref_properties = info.context.get("existing_ref_properties")
        for prop in validation_properties:
            if prop.is_valid:
                if prop.matching_reference_property not in existing_ref_properties:
                    raise ValueError(
                        f"""The predicted property {prop.property_name} 
                        was marked valid but the matching_reference_property {prop.matching_reference_property} 
                        does not exist in the reference knowledge base."""
                    )
        return validation_properties


class KnowledgeBase(BaseModel):
    entity_label: str
    properties: Dict[str, Any]


class EvaluationKB(BaseModel):
    predicted_knowledge_base: KnowledgeBase = Field(
        ...,
        description="The predicted knowledge base that must be evaluated against the reference."
    )
    reference_knowledge_base: KnowledgeBase = Field(
        ..., 
        description="The reference knowledge base used for evaluating prediction."
    )
    validated_kb: ValidatedKnowledgeBase = None


    @model_validator(mode="after")
    def eval_kb(self, context: str):

        existing_pred_properties = list(self.predicted_knowledge_base.properties.keys())
        existing_ref_properties = list(self.reference_knowledge_base.properties.keys())

        resp = client.chat.completions.create(
            response_model=ValidatedKnowledgeBase,
            messages=[
                {
                    "role": "system",
                    "content": f"""You are a knowledge base evaluator.""",
                },
                {
                    "role": "user",
                    "content": f"""Using your knowledge of the world and 
                    the given reference knowledge base, are the predicted properties and values valid
                    in the predicted knowledge base?
                    \nPredicted Knowledge Base: {self.predicted_knowledge_base}
                    \n\nReference Knowledge Base: {self.reference_knowledge_base}""",
                }
            ],
            validation_context={
                "existing_ref_properties": existing_ref_properties,
                "existing_pred_properties": existing_pred_properties,
            },
            max_retries=4,
            model=MODEL,
        )
        self.validated_kb = resp
            
        return self
    

# Evaluate!

In [ ]:
results = []
for idx in range(pred_kbs):
    # 🚨 NOTE: take the text data OUT! It'll fill the prompt otherwise
    chunked_content = filtered_ref_kbs[idx].pop('chunked_content', None) # take the text out 
    eval_info = {
        'predicted_knowledge_base': pred_kbs[idx],
        'reference_knowledge_base': filtered_ref_kbs[idx],
    }
    try:
        result = EvaluationKB(**eval_info)
        results.append(result)
    except Exception as e:
        print(f"Failed to evaluate at KB {idx} with error {e}")

In [70]:
# these should remain unchanged
print(results[0].model_dump()['predicted_knowledge_base'] == pred_kbs[0])
print(results[0].model_dump()['reference_knowledge_base'] == filtered_ref_kbs[0])

True
True


In [71]:
results[0].model_dump()['validated_kb']

{'entity_label': 'Alexei Navalny',
 'properties': [{'property_name': 'Name',
   'property_value': 'Alexei Navalny',
   'is_valid': True,
   'error_message': None,
   'matching_reference_property': 'name in native language'},
  {'property_name': 'Full Name',
   'property_value': 'Alexei Anatolyevich Navalny',
   'is_valid': True,
   'error_message': None,
   'matching_reference_property': 'given name'},
  {'property_name': 'Date of Birth',
   'property_value': '4 June 1976',
   'is_valid': True,
   'error_message': None,
   'matching_reference_property': 'date of birth'},
  {'property_name': 'Date of Death',
   'property_value': '16 February 2024',
   'is_valid': True,
   'error_message': None,
   'matching_reference_property': 'date of death'},
  {'property_name': 'Citizenship',
   'property_value': 'Russian',
   'is_valid': True,
   'error_message': None,
   'matching_reference_property': 'country of citizenship'},
  {'property_name': 'Occupation',
   'property_value': 'Politician and

In [93]:
def parse_none(s):
    if s is not None:
        return None if s.lower() in ['none', 'null'] else s
    return s

def calc_metrics(result: Dict, metrics: Dict[str, int] = None) -> Dict[str, int]:
    '''Calculate TP, FP, FN for a given result. Add to metrics dict if provided'''
    if metrics is None:
        tp = 0
        fp = 0
        fn = 0
    else:
        tp, fp, fn = metrics['tp'], metrics['fp'], metrics['fn']

    reference_properties_caught = []
    for val in result['validated_kb']['properties']:

        # true positive
        if val['is_valid'] is True:  
            tp += 1
            reference_properties_caught.append(val['matching_reference_property'])
        # false positive
        elif val['is_valid'] is False and (parse_none(val['matching_reference_property']) is None):
            fp += 1

    # false negative -- get all the reference properties that WEREN'T predicted
    for ref_prop_name in result['reference_knowledge_base']['properties'].keys():
        if ref_prop_name not in reference_properties_caught:
            fn += 1

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = (2 * (precision * recall)) / (precision + recall)

    print("----------")
    print(f"Validating for entity {result['validated_kb']['entity_label']}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1_score}")
    print("----------")
    return {'tp': tp, 'fp': fp, 'fn': fn}

In [94]:
metrics = calc_metrics(results[0].model_dump())

----------
Validating for entity Alexei Navalny
Precision: 0.4444444444444444
Recall: 0.21621621621621623
F1 Score: 0.2909090909090909
----------


In [ ]:
def visually_verify_results(result: Dict):
    '''Look at the validation KB, prediction KB, and reference KB side-by-side'''
    print(f"Validating for entity {result['validated_kb']['entity_label']}")

    for val in result['validated_kb']['properties']:
        print('\n--------------------------')
        print(f"🤖 Validated Property: {json.dumps(val, indent=4)}")

        matching_pred_property = result['predicted_knowledge_base']['properties'][val['property_name']]
        print(f"What does the prediction KB say? \n{matching_pred_property}")

        # if the predicted property was valid, get the reference the LLM points to
        if val['is_valid'] is True:  
            matching_ref_property = result['reference_knowledge_base']['properties'][val['matching_reference_property']]
            print(f"✅What does the reference KB say? \n{val['matching_reference_property']}: {matching_ref_property}")
        else:
            print(f"🚨The LLM said this one is not in the reference KB!")

In [87]:
visually_verify_results(results[0].model_dump())

Validating for entity Alexei Navalny

--------------------------
🤖 Validated Property: {
    "property_name": "Name",
    "property_value": "Alexei Navalny",
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "name in native language"
}
What does the prediction KB say? 
Alexei Navalny
✅What does the reference KB say? 
name in native language: ["{'text': 'Алексей Анатольевич Навальный', 'language': 'ru'}"]

--------------------------
🤖 Validated Property: {
    "property_name": "Full Name",
    "property_value": "Alexei Anatolyevich Navalny",
    "is_valid": true,
    "error_message": null,
    "matching_reference_property": "given name"
}
What does the prediction KB say? 
Alexei Anatolyevich Navalny
✅What does the reference KB say? 
given name: ['Alexey']

--------------------------
🤖 Validated Property: {
    "property_name": "Date of Birth",
    "property_value": "4 June 1976",
    "is_valid": true,
    "error_message": null,
    "matching_reference_p

In [ ]:
results_json = [r.model_dump() for r in results]
save_jsonl(results_json, '../../data/evaluation_results.jsonl')

In [ ]:
'''
Initial out for evaluation:

{'predicted_knowledge_base': {'entity_label': 'Alexei Navalny',
  'properties': {'Name': 'Alexei Navalny',
   'Full Name': 'Alexei Anatolyevich Navalny',
   'Date of Birth': '4 June 1976',
   'Date of Death': '16 February 2024',
   'Citizenship': 'Russian',
   'Occupation': 'Politician and Anti-corruption activist',
   'Founded Project': ['RosYama'],
   'Launched Project': ['Anti-Corruption Foundation (FBK)'],
   'Title': 'Investigation and Legal Cases',
   'Description': 'Details about investigations and legal cases involving Alexei Navalny.',
   'Date of death': '16 February 2024',
   'Place of death': 'Yamalo-Nenets in Western Siberia',
   'stated by': 'Alexei Kudrin, Boris Akunin, Mikhail Khodorkovsky, Vladimir Zhirinovsky, Marie Harf, Catherine Ashton, Andreas Schockenhoff, The New York Times, Alexei Venediktov, Levada Center, Leonid Volkov, Alexander Verkhovskiy',
   'criticised by': ['Vladimir Zhirinovsky',
    'United States Department of State Deputy Spokesperson Marie Harf',
    'European Union High Representative Catherine Ashton',
    "Andreas Schockenhoff, Germany's Commissioner for German-Russian Coordination",
    'Levada Center experts',
    'Alexander Verkhovskiy, head of the Moscow-based SOVA hate crimes monitor'],
   'supported by': 'Boris Nemtsov Foundation for Freedom, Geneva Summit for Human Rights and Democracy, Time magazine, Casimir Pulaski Foundation, European Parliament, M100 Media Award, Daniel Roher',
   'Family': ['Married to Yulia Abrosimova',
    'Had two children: daughter Darya (Dasha) Navalnaya and son Zakhar',
    'Daughter began undergraduate studies at Stanford University in September 2019'],
   'Residence': 'Lived primarily in a three-room apartment in Maryino District in southeast Moscow since 1998',
   'Religion': 'Became a member of the Russian Orthodox Church'}},
 'reference_knowledge_base': {'entity_label': 'Alexei Navalny',
  'properties': {'member of': ['Russian Opposition Coordination Council',
    'Yale World Fellows'],
   'sex or gender': ['male'],
   'educated at': ['Finance University under the Government of the Russian Federation',
    'Yale University',
    "Peoples' Friendship University of Russia",
    'Yale World Fellows'],
   'image': ['Alexey Navalny (cropped) 1.jpg'],
   'member of political party': ['Yabloko',
    'Progress Party',
    'Russia of the Future'],
   'Commons category': ['Alexey Navalny'],
   'employer': ['Anti-Corruption Foundation', 'Aeroflot'],
   'date of birth': ['+1976-06-04T00:00:00Z'],
   'religion or worldview': ['Eastern Orthodoxy'],
   'country of citizenship': ['Soviet Union', 'Russia'],
   'field of work': ['politics', 'jurisprudence'],
   'place of birth': ['Butyn'],
   'instance of': ['human'],
   'described by source': ['Lentapedia', 'Navalny'],
   'official website URL': ['https://navalny.com'],
   'given name': ['Alexey'],
   'significant event': ['Yves Rocher case',
    'poisoning of Alexei Navalny',
    'Kirovles trial',
    'incarceration',
    'poisoning of Alexei Navalny',
    'death of Alexei Navalny'],
   'sibling': ['Oleg Navalny'],
   'languages spoken, written or signed': ['Russian', 'English'],
   'name in native language': ["{'text': 'Алексей Анатольевич Навальный', 'language': 'ru'}"],
   'spouse': ['Yulia Navalnaya'],
   'ISNI': ['0000000377416015', '0000000410082812'],
   "topic's main category": ['Category:Alexei Navalny'],
   'on focus list of Wikimedia project': ['WikiProject Human Rights'],
   'family name': ['Navalny'],
   'social media followers': ['+2355803',
    '+2236418',
    '+6460000',
    '+2673187',
    '+2980696',
    '+6370000'],
   'position held': ['Leader of Russia of the Future',
    'Leader of Russia of the Future'],
   'award received': ['Person of the Year',
    'FP Top 100 Global Thinkers',
    'Prize of the Platform of European Memory and Conscience',
    'Time 100',
    'Gold Play Button',
    'Courage Award',
    'Boris Nemtsov Prize',
    'Knight of Freedom Award',
    'Sakharov Prize',
    'M100 Media Award',
    'The BOBs',
    'Silver Play Button',
    'Civil Courage Prize',
    'Günter Walraff award',
    'Bambi Award'],
   'height': ['+1.89'],
   'occupation': ['politician',
    'lawyer',
    'activist',
    'blogger',
    'public figure',
    'activist shareholder',
    'YouTuber',
    'film director',
    'announcer',
    'screenwriter',
    'entrepreneur',
    'political prisoner',
    'jurist',
    'prisoner of conscience'],
   'child': ['Darya Navalnaya', 'Zakhar Navalny'],
   'date of death': ['+2024-02-16T00:00:00Z'],
   'place of death': ['Corrective colony No. 3, YaNAO'],
   'described at URL': ['http://dbpedia.org/resource/Alexei_Navalny'],
   'mother': ['Lyudmila Navalnaya'],
   'father': ['Anatoly Navalny']}},
 'validated_kb': {'entity_label': 'Alexei Navalny',
  'properties': [{'property_name': 'Name',
    'property_value': 'Alexei Navalny',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'name in native language'},
   {'property_name': 'Full Name',
    'property_value': 'Alexei Anatolyevich Navalny',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'given name'},
   {'property_name': 'Date of Birth',
    'property_value': '4 June 1976',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'date of birth'},
   {'property_name': 'Date of Death',
    'property_value': '16 February 2024',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'date of death'},
   {'property_name': 'Citizenship',
    'property_value': 'Russian',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'country of citizenship'},
   {'property_name': 'Occupation',
    'property_value': 'Politician and Anti-corruption activist',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'occupation'},
   {'property_name': 'Founded Project',
    'property_value': ['RosYama'],
    'is_valid': False,
    'error_message': "The founded project 'RosYama' does not match any reference property.",
    'matching_reference_property': None},
   {'property_name': 'Launched Project',
    'property_value': ['Anti-Corruption Foundation (FBK)'],
    'is_valid': False,
    'error_message': "The launched project 'Anti-Corruption Foundation (FBK)' does not match any reference property.",
    'matching_reference_property': None},
   {'property_name': 'Title',
    'property_value': 'Investigation and Legal Cases',
    'is_valid': False,
    'error_message': "The title 'Investigation and Legal Cases' does not match any reference property.",
    'matching_reference_property': None},
   {'property_name': 'Description',
    'property_value': 'Details about investigations and legal cases involving Alexei Navalny.',
    'is_valid': False,
    'error_message': 'The description does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'Date of death',
    'property_value': '16 February 2024',
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'date of death'},
   {'property_name': 'Place of death',
    'property_value': 'Yamalo-Nenets in Western Siberia',
    'is_valid': False,
    'error_message': "The place of death 'Yamalo-Nenets in Western Siberia' does not match the reference property 'Corrective colony No. 3, YaNAO'.",
    'matching_reference_property': None},
   {'property_name': 'stated by',
    'property_value': ['Alexei Kudrin',
     'Boris Akunin',
     'Mikhail Khodorkovsky',
     'Vladimir Zhirinovsky',
     'Marie Harf',
     'Catherine Ashton',
     'Andreas Schockenhoff',
     'The New York Times',
     'Alexei Venediktov',
     'Levada Center',
     'Leonid Volkov',
     'Alexander Verkhovskiy'],
    'is_valid': True,
    'error_message': None,
    'matching_reference_property': 'described by source'},
   {'property_name': 'criticised by',
    'property_value': ['Vladimir Zhirinovsky',
     'United States Department of State Deputy Spokesperson Marie Harf',
     'European Union High Representative Catherine Ashton',
     "Andreas Schockenhoff, Germany's Commissioner for German-Russian Coordination",
     'Levada Center experts',
     'Alexander Verkhovskiy, head of the Moscow-based SOVA hate crimes monitor'],
    'is_valid': False,
    'error_message': 'The list of critics does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'supported by',
    'property_value': 'Boris Nemtsov Foundation for Freedom, Geneva Summit for Human Rights and Democracy, Time magazine, Casimir Pulaski Foundation, European Parliament, M100 Media Award, Daniel Roher',
    'is_valid': False,
    'error_message': 'The list of supporters does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'Family',
    'property_value': ['Married to Yulia Abrosimova',
     'Had two children: daughter Darya (Dasha) Navalnaya and son Zakhar',
     'Daughter began undergraduate studies at Stanford University in September 2019'],
    'is_valid': False,
    'error_message': 'The family information does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'Residence',
    'property_value': 'Lived primarily in a three-room apartment in Maryino District in southeast Moscow since 1998',
    'is_valid': False,
    'error_message': 'The residence information does not match any reference property.',
    'matching_reference_property': None},
   {'property_name': 'Religion',
    'property_value': 'Became a member of the Russian Orthodox Church',
    'is_valid': False,
    'error_message': 'The religion information does not match any reference property.',
    'matching_reference_property': None}]}}
'''